In [111]:
from aocd.models import Puzzle
from dataclasses import dataclass
import re

from enum import StrEnum, auto

In [426]:
puzzle = Puzzle(2024, 6)

input_data = puzzle.input_data
example_data = puzzle.examples[0].input_data

puzzle.url

'https://adventofcode.com/2024/day/6'

In [441]:
split_input = input_data.split()
split_input = example_data.split()

mydict = {}

for row_index, row in enumerate(split_input):
    for column_index, item in enumerate(row):
        mydict[(row_index, column_index)] = item


In [474]:
class Direction(StrEnum):
    UP = "^"
    DOWN = "v"
    LEFT = "<"
    RIGHT = ">"

@dataclass
class Position:
    row: int
    column: int

    def __str__(self):
        return f"({self.row}, {self.column})"

def fuck_this_shit(lst: list[Position]) -> str:
    bajs = ""
    for item in lst:
        bajs = bajs + str(item)
    return bajs


@dataclass
class Grid: 
    grid: dict[tuple[int, int]: str]
    rows: int | None = None
    columns: int | None = None
    guard_position: Position | None = None
    guard_direction: Direction | None = None
    traveled: dict[int: list[Position]] | None = None
    turns: int = 0

    def __post_init__(self):
        self.rows = max(self.grid)[0]
        self.columns = max(self.grid, key=lambda x: x[1])[1]
        self.traveled = {0: []}

    def get_guard_position(self):
        position = [key for key in self.grid if self.grid[key] in ["^", "<", ">", "v"]][0]
        return Position(position[0], position[1])
    
    def get_guard_direction(self) -> Direction:
        return Direction(self.get_value(self.guard_position))
    
    def get_neighbour(self, direction: Direction, position:Position):
        return {Direction.UP: self.grid.get((position.row-1, position.column)), 
                Direction.LEFT: self.grid.get((position.row, position.column-1)), 
                Direction.DOWN: self.grid.get((position.row+1, position.column)), 
                Direction.RIGHT: self.grid.get((position.row, position.column+1))
                }.get(direction)
    
    def get_value(self, position:Position) -> str:
        return self.grid.get((position.row, position.column))
    
    def mark_position_X(self, position: Position):
        self.grid[(position.row, position.column)] = "X"
    
    def turn_guard_right(self):
        self.guard_direction = {Direction.UP: Direction.RIGHT,
                                Direction.RIGHT: Direction.DOWN, 
                                Direction.DOWN: Direction.LEFT, 
                                Direction.LEFT: Direction.UP}.get(self.guard_direction)
        self.turns = self.turns +1
        self.traveled[self.turns] = [self.guard_position]

    def move_guard_forward(self) -> str:
        moveto = {Direction.UP: Position(self.guard_position.row-1, self.guard_position.column), 
                               Direction.RIGHT: Position(self.guard_position.row, self.guard_position.column + 1), 
                               Direction.DOWN: Position(self.guard_position.row +1, self.guard_position.column), 
                               Direction.LEFT: Position(self.guard_position.row, self.guard_position.column -1)}.get(self.guard_direction)
        
        if moveto.row not in range(0, self.rows+1) or moveto.column not in range(0, self.columns+1):
            return "out of bounds"
        
        self.guard_position = {Direction.UP: Position(self.guard_position.row-1, self.guard_position.column), 
                               Direction.RIGHT: Position(self.guard_position.row, self.guard_position.column + 1), 
                               Direction.DOWN: Position(self.guard_position.row +1, self.guard_position.column), 
                               Direction.LEFT: Position(self.guard_position.row, self.guard_position.column -1)}.get(self.guard_direction)
        
        current_stretch = self.traveled.get(self.turns)
        current_stretch.append(self.guard_position)
        
        self.traveled[self.turns] = current_stretch
        
        return "moved"
    
    def test_for_loop(self, position:Position, direction:Direction) -> bool: 
        
        testgrid = Grid(self.grid.copy())
        testgrid.guard_position = position
        testgrid.guard_direction = direction
        testgrid.traveled = self.traveled
        testgrid.turns = self.turns

        testgrid.turn_guard_right()
        status = ""

        path = testgrid.get_neighbour(testgrid.guard_direction, testgrid.guard_position)

        while status != "out of bounds":

            while path == "#":
                testgrid.turn_guard_right()
                path = testgrid.get_neighbour(testgrid.guard_direction, testgrid.guard_position)

            while path != "#" and status != "out of bounds":
                testgrid.mark_position_X(testgrid.guard_position)
                status = testgrid.move_guard_forward()
                path = testgrid.get_neighbour(testgrid.guard_direction, testgrid.guard_position)
                
            testgrid.mark_position_X(testgrid.guard_position)

            if status == "out of bounds":
                return False

            current_path = fuck_this_shit(testgrid.traveled[testgrid.turns])

            gone_two_steps = len(testgrid.traveled[testgrid.turns]) > 1
            visited_before = sum(1 for value in testgrid.traveled.values() if current_path in fuck_this_shit(value)) > 1

            if gone_two_steps and visited_before:
                return True

        return False


    def __str__(self) -> str:
        mystring = ""
        for row in range(self.rows+1):
            for column in range(self.columns+1):
                mystring = mystring + self.grid.get((row, column))
            mystring = mystring + "\n"
        return mystring


In [475]:
griddict = mydict.copy()

mygrid = Grid(griddict)
mygrid.guard_position = mygrid.get_guard_position()
mygrid.guard_direction = mygrid.get_guard_direction()
status = "ok"

path = mygrid.get_neighbour(mygrid.guard_direction, mygrid.guard_position)
while status != "out of bounds":
    while path == "#":
        mygrid.turn_guard_right()
        path = mygrid.get_neighbour(mygrid.guard_direction, mygrid.guard_position)

    while path != "#" and status != "out of bounds":
        mygrid.mark_position_X(mygrid.guard_position)
        status = mygrid.move_guard_forward()
        path = mygrid.get_neighbour(mygrid.guard_direction, mygrid.guard_position)

count = sum([1 for value in mygrid.grid.values() if value == "X"])
print(count)

41


In [476]:
puzzle.answer_a = count

aocd will not submit that answer. At 2024-12-06 14:10:53.585267-05:00 you've previously submitted 4665 and the server responded with:
That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]
It is certain that '41' is incorrect, because '41' != '4665'.


In [477]:
griddict = mydict.copy()
mygrid = Grid(griddict)

mygrid.guard_position = mygrid.get_guard_position()
mygrid.guard_direction = mygrid.get_guard_direction()

status = "ok"
loop_count = 0
inner_loop_count = 0

path = mygrid.get_neighbour(mygrid.guard_direction, mygrid.guard_position)

while status != "out of bounds":

    while path == "#":
        mygrid.turn_guard_right()
        path = mygrid.get_neighbour(mygrid.guard_direction, mygrid.guard_position)

    inner_loop_count = 0
    while path != "#" and status != "out of bounds":
        mygrid.mark_position_X(mygrid.guard_position)
        
        loop = mygrid.test_for_loop(mygrid.guard_position, mygrid.guard_direction)
        
        if loop:
            inner_loop_count = inner_loop_count + 1

        status = mygrid.move_guard_forward()
        path = mygrid.get_neighbour(mygrid.guard_direction, mygrid.guard_position)
    
    path = mygrid.get_neighbour(mygrid.guard_direction, mygrid.guard_position)
    
    loop_count = loop_count + inner_loop_count
    
print(loop_count)

6


In [440]:
puzzle.answer_b = loop_count

aocd will not submit that answer again. At 2024-12-07 16:16:03.839566-05:00 you've previously submitted 3267 and the server responded with:
That's not the right answer; your answer is too high.  If you're stuck, make sure you're using the full input data; there are also some general tips on the about page, or you can ask for hints on the subreddit.  Please wait one minute before trying again. [Return to Day 6]


In [433]:
[1,2] in [1,2,3]

False